In [96]:
from hashlib import sha256
import json
import time


#block

class block:
    def __init__(self, index, transaction, timestample, prev_hash):
        self.index = index
        self.transaction = transaction
        self.timestample = timestample
        self.prev_hash = prev_hash
        
    def compute_hash(self):
    
        block_string = json.dumps(self.__dict__, sort_keys=True)
        return sha256(block_string.encode()).hexdigest()

class transaction:
    def __init__(self, SN, from_key, to_key, validation):
        self.SN = SN
        self.from_key = from_key
        self.to_key = to_key
        self.validation = validation    


chain = dict()
chain['b0'] = block(0,transaction('sis',"gene","n",True),0,0)   #Genesis



def send(from_key, to_key, SN):                   
    if SN in from_key.parts:
        from_key.parts.pop(from_key.parts.index(SN))
        to_key.parts.append(SN)
        print('part sent')
        
        key = 'b{}'.format(len(chain))
        chain[key] = block(len(chain),transaction(SN, from_key, to_key, True),'time',hash(str(chain['b{}'.format(len(chain)-1)].transaction.from_key)+str(chain['b{}'.format(len(chain)-1)].transaction.SN)))        
        #para recuperar os dados cache["ab123"].SN/from_key...
        
        
    
    elif SN not in from_key.parts:
        print('Part not in your wallet')
    
    
class wallet:
    def __init__(self,parts,public_key,private_key):
        self.parts = parts
        self.public_key = public_key
        self.private_key = private_key

    import random
    import string
    
    def new_wallet(username):    # 
        username = wallet([],(''.join(random.choices(string.ascii_uppercase + string.digits, k=8))),123)
        
    
def check_chain():
    last_block = -1    
    while len(chain) + last_block != 0:
        previous_hash = hash(str(chain['b{}'.format(len(chain)+last_block-1)].transaction.from_key)+str(chain['b{}'.format(len(chain)+last_block-1)].transaction.SN))
        if chain['b{}'.format(len(chain)+last_block)].prev_hash == previous_hash:
            last_block = last_block - 1
            
        else:
            print ('Block {} invalid'.format(len(chain)+last_block))
            last_block = last_block - 1
    else:
        print('finish')
        
    #criar variável "chain status" e não permitir transações quando estiver False
        
        
        
        #criar sistema de explorer
        #ex: hist_part(SN)


In [105]:
check_chain()

Block 4 invalid
Block 3 invalid
Block 1 invalid
finish


In [98]:
a = wallet([123],'a','a1')
b = wallet([],'b','b1')
c = wallet([],'c','c1')
d = wallet([],'d','d1')
e = wallet([],'e','e1')
send(a,b,123)
send(b,c,123)
send(c,d,123)
send(d,e,123)
chain

part sent
part sent
part sent
part sent


{'b0': <__main__.block at 0x7fa820142c18>,
 'b1': <__main__.block at 0x7fa820132ac8>,
 'b2': <__main__.block at 0x7fa820132b00>,
 'b3': <__main__.block at 0x7fa820132080>,
 'b4': <__main__.block at 0x7fa820132278>}

In [104]:
chain['b1'].prev_hash =2